### 문제 1-2: 2단계 체인 만들기 - 영화 추천 시스템 해답

## 환경 설정

In [ ]:
# 필요한 라이브러리 설치
#%pip install -q langchain langchain-openai python-dotenv

In [ ]:
# 환경 변수 설정
from dotenv import load_dotenv
import os

# .env 파일에서 API 키 로드
load_dotenv()

## 문제 2 해답 구현

### 요구사항
1. 1단계 체인: 장르를 입력받아 영화 1편 추천
2. 2단계 체인: 추천받은 영화의 3줄 줄거리 요약 제공
3. ChatPromptTemplate 사용
4. 두 체인을 LCEL로 연결
5. 각 단계의 결과를 모두 출력하여 과정 확인

In [ ]:
# 필요한 라이브러리 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1단계: 장르별 영화 추천 프롬프트
movie_recommendation_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 전문가입니다. 사용자가 요청한 장르에 맞는 영화 1편을 추천하고 간단한 설명을 제공해주세요."),
    ("user", "{genre} 장르의 한국영화 1편을 추천해주세요. 영화 제목과 왜 이 영화를 추천하는지 이유도 함께 알려주세요.")
])

In [ ]:
# 2단계: 영화 줄거리 요약 프롬프트
plot_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 줄거리 요약 전문가입니다. 영화의 핵심 내용을 3줄로 간결하게 요약해주세요."),
    ("user", "다음 영화 추천 내용을 바탕으로 해당 영화의 등장인물과 줄거리를 정확히 10줄로 요약해주세요: {movie}")
])

In [ ]:
# LLM 모델 초기화
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 1단계 체인: 영화 추천
recommendation_chain = movie_recommendation_prompt | llm | StrOutputParser()
# Step 1: 사용자가 입력한 장르에 따라 영화 추천
movie = recommendation_chain.invoke({"genre": "Drama"})  # 영화 제목 얻기
print(" 추천된 영화:", movie)  # movie 값 출력

# 2단계 체인: 줄거리 요약 (1단계 결과를 입력으로 받음)
plot_chain = (
    {"movie": recommendation_chain}  # 1단계 체인의 결과를 movie로 전달
    | plot_summary_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 테스트 실행
try:
    genre = "Drama"
    
    print("영화 추천 시스템")
    print("==================")
    print(f"\n입력 장르: {genre}")
    
    # 1단계 실행 및 결과 출력
    movie_recommendation = recommendation_chain.invoke({"genre": genre})
    print("\n1단계 - 영화 추천 결과:")
    print(movie_recommendation)
    
    print("\n---\n")
    
    # 2단계 실행 및 결과 출력
    plot_summary = plot_chain.invoke({"genre": genre})
    print("2단계 - 영화 줄거리 요약:")
    print(plot_summary)
    
except Exception as e:
    print(f"오류 발생: {e}")

## 추가 테스트

In [ ]:
# 다른 장르로 추가 테스트
genre2 = "로맨스"

movie_recommendation2 = recommendation_chain.invoke({"genre": genre2})
plot_summary2 = plot_chain.invoke({"genre": genre2})

print(f"다른 장르로 테스트 - {genre2}:")
print("===============================")
print("\n영화 추천:")
print(movie_recommendation2)
print("\n줄거리 요약:")
print(plot_summary2)

## 체인 연결 패턴 설명

In [ ]:
# 체인 연결 방식 설명
print("체인 연결 방식 설명:")
print("====================")
print()
print("1단계 체인:")
print("movie_recommendation_prompt | llm | StrOutputParser()")
print()
print("2단계 체인:")
print('{"movie": recommendation_chain} | plot_summary_prompt | llm | StrOutputParser()')
print()
print("핵심 포인트:")
print("- 1단계 체인의 출력이 2단계 체인의 입력으로 자동 전달")
print("- movie 키를 통해 데이터 흐름 제어")
print("- LCEL을 통해 직관적인 파이프라인 구성")

## 학습 정리

### 문제 2에서 배운 핵심 개념
1. **ChatPromptTemplate**: 시스템과 사용자 메시지 구조화
2. **다단계 체인**: 여러 단계로 나누어 복잡한 작업 처리
3. **체인 간 데이터 전달**: {"key": previous_chain} 패턴
4. **단계별 결과 확인**: 각 체인의 중간 결과 출력

### 구현 완료 사항
- ✅ 2단계 체인 구현
- ✅ ChatPromptTemplate 사용
- ✅ 체인 간 데이터 전달
- ✅ 영화 추천 기능
- ✅ 줄거리 요약 기능
- ✅ 각 단계별 결과 출력
- ✅ 다양한 장르 테스트

### 체인 연결의 핵심 패턴
```python
# 기본 패턴
chain1 = prompt1 | llm | parser
chain2 = {"input_key": chain1} | prompt2 | llm | parser
```

이 패턴을 통해 복잡한 작업을 단계별로 분해하여 처리할 수 있습니다.